# Multivariate Time Series Forecasting with LSTMs in Keras
#### Reference: https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

### Train On Single Lag Timestep - All meters

In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
 


Using TensorFlow backend.


In [2]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 


In [3]:
# load dataset
#dataset = read_csv('DFS04690650Rand.txt', header=0, index_col=0)
dataset = read_csv('WorthingtonAllDropped.txt', header=0, sep='[\t]', parse_dates=True, squeeze=True, dayfirst=True, engine='python') 
values = dataset.values
dataset.dtypes
print(dataset.head())



         Temp  MeterNo  NoBeds  Area_m2  Const_Year  kWh  Month  Weekday  \
691403   1.35        4       2     87.8        2017    1      6        1   
336765   7.00       40       1     69.4        2017    0     10        1   
72500   10.65       14       1     64.6        2017    0     10        4   
759515  14.35        3       2     87.8        2017    0      8        1   
895578   9.30       28       2     85.2        2017    0     12        2   

        Seconds  Daytime  Holiday  
691403    31500        1        0  
336765    10800        0        0  
72500     41400        1        0  
759515    54900        1        0  
895578     3600        0        0  


In [4]:
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
reframed.dtypes
# drop columns we don't want to predict
reframed.drop(reframed.columns[[11,12,13,14,15,17,18,19,20,21]], axis=1, inplace=True)
print(reframed.head())

   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)  var6(t-1)  \
1   0.241319       0.06        1.0   0.775342        0.0      0.125   
2   0.437500       0.78        0.0   0.271233        0.0      0.000   
3   0.564236       0.26        0.0   0.139726        0.0      0.000   
4   0.692708       0.04        1.0   0.775342        0.0      0.000   
5   0.517361       0.54        1.0   0.704109        0.0      0.000   

   var7(t-1)  var8(t-1)  var9(t-1)  var10(t-1)  var11(t-1)  var6(t)  
1   0.454545   0.166667   0.361702         1.0         0.0      0.0  
2   0.818182   0.166667   0.117021         0.0         0.0      0.0  
3   0.818182   0.666667   0.478723         1.0         0.0      0.0  
4   0.636364   0.166667   0.638298         1.0         0.0      0.0  
5   1.000000   0.333333   0.031915         0.0         0.0      0.0  


In [5]:
# split into train and test sets
values = reframed.values
n_train_periods = int(len(values)*0.8)
train = values[:n_train_periods, :]
test = values[n_train_periods:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
 


(792380, 1, 11) (792380,) (198095, 1, 11) (198095,)


In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=32, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
#pyplot.show()
 


Train on 792380 samples, validate on 198095 samples
Epoch 1/50
 - 102s - loss: 0.0119 - val_loss: 0.0115
Epoch 2/50
 - 101s - loss: 0.0116 - val_loss: 0.0115
Epoch 3/50
 - 99s - loss: 0.0116 - val_loss: 0.0116
Epoch 4/50
 - 100s - loss: 0.0116 - val_loss: 0.0115
Epoch 5/50
 - 97s - loss: 0.0116 - val_loss: 0.0116
Epoch 6/50
 - 96s - loss: 0.0116 - val_loss: 0.0117
Epoch 7/50
 - 98s - loss: 0.0116 - val_loss: 0.0115
Epoch 8/50
 - 98s - loss: 0.0116 - val_loss: 0.0116
Epoch 9/50
 - 101s - loss: 0.0116 - val_loss: 0.0115
Epoch 10/50
 - 105s - loss: 0.0116 - val_loss: 0.0116
Epoch 11/50
 - 92s - loss: 0.0116 - val_loss: 0.0116
Epoch 12/50
 - 89s - loss: 0.0116 - val_loss: 0.0115
Epoch 13/50
 - 88s - loss: 0.0116 - val_loss: 0.0115
Epoch 14/50
 - 89s - loss: 0.0116 - val_loss: 0.0116
Epoch 15/50
 - 89s - loss: 0.0116 - val_loss: 0.0115
Epoch 16/50
 - 86s - loss: 0.0116 - val_loss: 0.0117
Epoch 17/50
 - 90s - loss: 0.0116 - val_loss: 0.0115
Epoch 18/50
 - 89s - loss: 0.0116 - val_loss: 0.011

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)